# Deep Learning Project 3 due Thursday 3/31

## Convolutional Neural Networks and Image Classification

In this project, you will apply convolutional neural networks and ideas learned in Chapters 4 and 5 of our textbook to classify images in a dataset of your choice. 

### The datasets

There are three other image datasets packaged with Keras that we have not yet explored: 
* CIFAR-10: is a dataset of 50,000 32x32 color training images and 10,000 test images, labeled over 10 categories.
* CIFAR-100: is a dataset of 50,000 32x32 color training images and 10,000 test images, labeled over 100 fine-grained classes that are grouped into 20 coarse-grained classes. 
* Fashion MNIST dataset: is a dataset of 60,000 28x28 grayscale images of 10 fashion categories, along with a test set of 10,000 images

Instructions for loading these datasets from Keras are found [here](https://keras.io/api/datasets/). 

**CIFAR-10 and Fashion MNIST are pretty simple to solve with CNNs. So use these datasets only for experimentation purposes.**

For Project 2, you can choose CIFAR-100 or another datasets of your own choice for multiclass classification. Just be mindful of the problem size (number of samples) you are trying to tackle. [Kaggle](kaggle.com) is a good place to look around. One clean Kaggle project I know of is the [Humpback whale dataset](https://www.kaggle.com/c/humpback-whale-identification/data?select=train). There are roughly 25.4K files in the training set and 7.9K files in the test set. This dataset  has received a lot of attention over the last two years. 

### The Approach
Section 4.5 of the textbook outlines a clear workflow for solving a machine-learning problem. As a reminder, the steps included the following:
1. Defining the problem and assembling a dataset
2. Choosing a measure of success
3. Deciding on an evaluation protocol
4. Preparing the data
5. Developing a model that does better than a baseline
6. Scaling up: developing a model that overfits
7. Regularizing the model and tuning hyper parameters

For this project, you are required to follow the above workflow, documenting your work along the way in your report. For example, for the Humpback whale dataset, I would spend some time looking at the dataset to determine the number of classes, the number of samples per class, the number of pixels per image, etc. One key thing would be to decide the size of the images for my network given the computing power available to me.

### Requirements
Here are requirements for your work:
* You must design and attempt to train your own convolutional neural network (implemented in Keras), following the workflow of section 4.5. 
* You must implement transfer learning (section 5.3) either through feature extraction or fine tuning. Choose one of the models prepackaged with Keras (Xception, VGG16, etc). 
* You must use one of the three visualization techniques of section 5.4 to attempt to interpret your results.

### Timeline/What to turn in for Project 3

Suggested timeline:
* Tuesday 3/22: Choose a dataset and have a model that beats a baseline.
* Thursday 3/24: Have successfully attempted transfer learning. 
* Tuesday 3/28: Have completed work to solve overfitting.
* Thursday 3/31: Have completed visualization and interpretation.

**Your report for Project 3 (a Jupyter notebook with your code, comments in markup cells, and output cells) is due Thursday 3/31.**  

### Learning outcomes
After completing project 3, you will be able to:
* Implement convolutional neural networks for image classification problems
* Organize datasets and apply a workflow to solve a machine learning problem
* Make decisions regarding input data, model architecture, etc. that affect the solution of classification problems